<a href="https://colab.research.google.com/github/DomingosLemos/Deploy-cnn-test/blob/main/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inception

# Import the necessary libraries

In [4]:
!pip install split-folders
import splitfolders
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow import keras
from tensorflow.keras import models
import cv2
from sklearn.metrics import confusion_matrix,classification_report

#models
from keras.applications.inception_v3 import InceptionV3

### Mount the google drive where we have a zip with all the images

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Unzip images to folders

In [2]:
!7z x /content/drive/MyDrive/EDSA/images.zip -o/content/images


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/EDSA/                                      1 file, 5132666035 bytes (4895 MiB)

Extracting archive: /content/drive/MyDrive/EDSA/images.zip
  4% 4096 Open               36% 36864 Open                60% 61440 Open               --
Path = /content/drive/MyDrive/EDSA/images.zip
Type = zip
Physical Size = 5132666035
64-bit = +

  0%      0% 635 - apple_pie/3211704.jpg                                  1% 1135 - baby_back_ribs/1480327.jpg                          

### Split images with splitfolder
Put on:
* content/output/train with 70% (700 images for each folder)
* content/output/val with 20% (200 images for each folder)
* content/output/test with 10% (100 images for each folder)


In [5]:
splitfolders.ratio('/content/images', output="output", seed=1337, ratio=(.7, .2, .1)) 

Copying files: 101000 files [02:17, 735.39 files/s]


## Inception model

As we have a small dataset for each category (700 images) we will use "Image Augmentation" to increase the size of our dataset a little

In [ ]:
#Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

### Get the data

In [ ]:
#n_classes = 101
img_width, img_height = 299, 299 # Standard shape for pre-trained model
train_data_dir = '/content/output/train'
validation_data_dir = '/content/output/val'
nb_train_samples = 70700
nb_validation_samples = 20200
batch_size = 16

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
#Build the model
inception = InceptionV3(weights='imagenet', include_top=False)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(101,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)

# Compile model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)

# Fit model
history = model.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[checkpointer])

model.save('model_trained_3class.hdf5')

Found 70700 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.
87916544/87910968 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
4418/4418 [==============================] - 1875s 422ms/step - loss: 5.0553 - accuracy: 0.0407 - val_loss: 3.9280 - val_accuracy: 0.2904

Epoch 00001: val_loss improved from inf to 3.92801, saving model to best_model_3class.hdf5
Epoch 2/30
4418/4418 [==============================] - 1831s 415ms/step - loss: 3.8302 - accuracy: 0.2761 - val_loss: 2.6612 - val_accuracy: 0.5042

Epoch 00002: val_loss improved from 3.92801 to 2.66116, saving model to best_model_3class.hdf5
Epoch 3/30
4418/4418 [==============================] - 1783s 403ms/step - loss: 2.9322 - accuracy: 0.4278 - val_loss: 2.1312 - val_accuracy: 0.6059

Epoch 00003: val_loss improved from 2.66116 to 2.13121, saving model to best_model_3class.hdf5
Epoch 4/30
4418/4418 [==============================] - 1758s 398ms/step - loss: 2.4642 - accuracy: 0.5217 - val_loss: 1.8449 - val_accuracy: 0.6603

Epoch 00004: val_loss improved from 2.13121 to 1.84488, saving model to best_model_3class.hdf5
Epoch 5/30
4418/4418 [==

## Evaluating the Inception Model

In [6]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

batch_size = 16
img_width, img_height = 299, 299

test_generator=test_datagen.flow_from_directory(directory="/content/output/test/", 
                                                  batch_size=batch_size,
                                                  shuffle=False, #Shuffle was changed to false in order to ensure that the images were predicted in the correct order
                                                  class_mode="categorical",
                                                  target_size=(img_width, img_height),
                                                  seed=42)


Found 10100 images belonging to 101 classes.


In [12]:
#rename file 
!cp /content/drive/MyDrive/EDSA/model_trained_3class.hdf5 /content/drive/MyDrive/EDSA/best_model.h5

In [13]:
#Load model saved
 
Best_Model = load_model('/content/drive/MyDrive/EDSA/best_model.h5')

In [16]:
#https://stackoverflow.com/questions/53929819/generating-confusion-matrix-in-keras-for-multiclass-classification


predictions = Best_Model.predict_generator(test_generator)

y_pred = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())   

print(class_labels)

print(confusion_matrix(test_generator.classes, y_pred))

report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou